## Example: `Hull` American `put` Option Premium Calculation
A binomial lattice model assumes that at each discrete time increment, the state of the system, e.g., the share price of equity, can either increase by a factor $u$ with probability $p$ or decrease by a factor $d$ with probability $(1-p)$ in the next time interval. Thus, each discrete time interval can be modeled as a [Bernoulli random variable](https://en.wikipedia.org/wiki/Bernoulli_distribution):

<div>
    <center>
        <img src="figs/Fig-Binomial-Lattice-Schematic.svg" width="280"/>
    </center>
</div>

while each level (time slice) of the tree is described by a [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution). Different models have been developed to compute the tuple $(u,d,p)$. However, for now, let's specify these values, and calculate the value of an American `put` contract written with respect to the lattice.

## Learning objectives
The objective of `Lab 6c` is to familiarize students with computing future share price and the value of American `put` and `call` contracts using a [Binomial lattice approximation](https://en.wikipedia.org/wiki/Binomial_options_pricing_model). 

### Tasks 
* __Task 1__: Setup and populate an example lattice from Chapter 13 of `Hull`. 
    * `TODO`: Inspect the populated lattice’s `connectivity`, `levels`, and `data` fields. Can you explain these data?
* __Task 2__: Compute the premium of an American `put` contract, given values for the $(u,d,p)$ tuple
    * `TODO`: Compare the reported value for the `put` premium with the value reported by `Hull`. 
* __Task 3__ : Explore the `{excercise,hold}` decision that occurs at each node in the lattice, and the role of the `intrinsic` and `extrinsic` values.

## Setup
We load the [VLQuantitativeFinancePackage.jl](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl) package, and several other external [Julia](https://julialang.org/downloads/https://julialang.org/downloads/) packages, as well as some helper code, by calling the [include(...)](https://docs.julialang.org/en/v1/manual/code-loading/https://docs.julialang.org/en/v1/manual/code-loading/) command on the file `Include.jl`:

In [10]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Examples-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Examples-F23/Manifest.toml`


### Types
`Include.jl` loads some [problem-specific types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that will be helpful for the lattice model simulation of equity share prices:

The `MyBinomialEquityPriceTree` encodes the lattice model and has the fields:
    
* The `connectivity::Union{Nothing, Dict{Int64, Array{Int64,1}}}` field holds the indexes of the children for each parent node of the tree.
* The `levels::Union{Nothing, Dict{Int64, Array{Int64,1}}}` field holds indexes of nodes that belong to each level of the tree, i.e., a trading day.
* The `u::Float64` field holds the value of the `up` factor
* The `d::Float64` field holds the value of the `down` factor
* The `p::Float64` field holds the value of the probability of an `up` move.
* The `data::Union{Nothing, Dict{Int64, MyBiomialLatticeEquityNodeModel}}` holds each node in the tree; nodes are of type `MyBiomialLatticeEquityNodeModel`

Each node in the tree is a `MyBiomialLatticeEquityNodeModel` type, which has two important fields:
* The `price::Float64` field holds the price value for a node
* The `probability::Float64` field holds the probability value associated with this node

The `MyAmericanPutContractModel` and `MyAmericanCallContractModel`types hold data associated with American `put` (or `call`) contracts. In this lab, we care about the fields:
* The `K::Float64` field holds the strike price of the contract
* The `DTE::Union{Nothing, Float64}` field holds the duration of the contract (units: years)
* The `sense::Union{Nothing, Int64}` field holds the orientation of the contract (in this lab, `sense = 1`). 

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/):
  
`function build(model::Type{MyBinomialEquityPriceTree}, data::NamedTuple) -> MyBinomialEquityPriceTree` 
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the $(u,d,p)$ values) and returns an instance of the `MyBinomialEquityPriceTree` [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types). Note: the `MyBinomialEquityPriceTree` returned from the `build(...)` function does not have price or probability information computed yet. Call the `populate(…)` function to populate this data.

`function populate(model::MyBinomialEquityPriceTree, Sₒ::Float64, h::Int) -> MyBinomialEquityPriceTree`
> The `populate(...)` function takes the `model::MyBinomialEquityPriceTree` instance returned from `build(...)`, a starting share price $S_{o}$ and the height of the tree, i.e., the number of time steps to simulate, and returns an updated `model::MyBinomialEquityPriceTree` instance with the price and probabilities computed for each node in the tree.

`function build(model::Type{MyAmericanPutContractModel}, data::NamedTuple) -> MyAmericanPutContractModel`
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument, the strike price `K`, the duration of the contract `DTE` and the `sense = 1` flag, and returns an instance of a `MyAmericanPutContractModel` model. A similar method is also provided to construct `MyAmericanCallContractModel` instances. 

`function premium(contract::T, model::MyBinomialEquityPriceTree; choice::Function=_rational) -> Float64 where {T<:AbstractContractModel}`
> The `premium(...)` function takes the `contract::T` and  `model::MyBinomialEquityPriceTree` arguments and returns the `premium::Float64` (price) of the options contract, where the type `T` is any contract type `{call,put}`.

## Task 1: Setup and Populate Example Lattice From Hull
Let’s start by calculating the hypothetical share price of a stock that was reproduced from Chapter 13 of Hull. The lattice in this example has three levels (`L=0,1,2`), with an initial share price of $S_{\circ}=50.0$ USD (at the root). An `up` move has a probability of `p = 0.6523` and a magnitude of `u = 1.2`, while a `down` move has a magnitude of  `d = 0.8`.

### Implementation
Calculating the future share price with a binomial lattice requires setting the model’s parameters, including the initial share price (as a `Float64`), the number of time steps to simulate into the future, which equals the number levels of the tree starting from zero (as an `Int64`), the up and down move magnitudes (as `Float64` values), and the probability of an up move (as a `Float64`):

In [6]:
Sₒ,h,u,d,p,r̄,K,DTE = 50.0,2,1.2,0.8,0.6282,0.05,52.0,2;

Once these values are set, we use the `build(…)` function to create an empty lattice model of type `MyBinomialEquityPriceTree`, which is then passed to the `populate(…)` function using the [Julia piping operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) `|>`. The `populate(…)` function calculates the prices and probabilities of each node (type `MyBiomialLatticeEquityNodeModel`) in the tree:

In [4]:
hull_lattice_model = VLQuantitativeFinancePackage.build(MyBinomialEquityPriceTree, (
        u = u, d = d, p = p, μ = r̄)) |> (x-> populate(x, Sₒ = Sₒ, h = h));

In [15]:
hull_lattice_model.levels

Dict{Int64, Vector{Int64}} with 3 entries:
  0 => [0]
  2 => [3, 4, 5]
  1 => [1, 2]

In [16]:
hull_lattice_model.data

Dict{Int64, MyBiomialLatticeEquityNodeModel} with 6 entries:
  0 => MyBiomialLatticeEquityNodeModel(50.0, 1.0, 2.0, 5.08936)
  4 => MyBiomialLatticeEquityNodeModel(48.0, 0.46713, 4.0, 4.0)
  5 => MyBiomialLatticeEquityNodeModel(32.0, 0.138235, 20.0, 20.0)
  2 => MyBiomialLatticeEquityNodeModel(40.0, 0.3718, 12.0, 12.0)
  3 => MyBiomialLatticeEquityNodeModel(72.0, 0.394635, 0.0, 0.0)
  1 => MyBiomialLatticeEquityNodeModel(60.0, 0.6282, 0.0, 1.41467)

## Task 2: Compute the premium of American `put` contract
Now that we have the share price lattice, let's compute the the premium $\mathcal{P}_{p}$ of a American `put` contract using `backward induction` on the binomial lattice. The `put` contract has a strike price `K = 52` USD/share and a Days to Expiration `DTE` of `2-years`. 
* The value computed by Hull for the American option is $\mathcal{P}_{p} = 5.0894$ USD/share:

In [5]:
price_computed_by_Hull = 5.0894; # units: USD/share

In [7]:
american_put_contract_model = VLQuantitativeFinancePackage.build(MyAmericanPutContractModel, (
        K = K, DTE = DTE, sense = 1));

Now, we call the `premium(...)` method with the contract, and the lattice, as parameters. The `premium(...)` method returns the premium $\mathcal{P}_{p}$ (price) of the contract. We store this value in the `my_put_premium` variable:

In [8]:
my_put_premium = premium(american_put_contract_model, hull_lattice_model);

### Check: Do we get the correct contract price?

In [9]:
@assert isapprox(price_computed_by_Hull, my_put_premium, rtol=1e-4)

## Task 3: Inspect the Lattice nodes
The nodes in the `hull_lattice_model`, which can be accessed using the `data` field, contain information about the `instrinsic` and `extrinsic` values of the `put` option contract. When dealing with American options contracts, it is important to understand these two types of values: 

* `Intrinsic` value is the difference between the current market price of the underlying asset and the option’s strike price, indicating the immediate value that can be obtained by exercising the option at the current moment. 
* `Extrinsic` value, on the other hand, is the portion of the option’s premium that is not related to its intrinsic value. It represents the premium paid for the potential price movement of the underlying asset over the remaining time until the option’s expiration, as well as other factors such as implied volatility.